In [420]:
import pandas as pd
import requests
import tweepy
import yaml 
import os
import json
import numpy as np

# Read PitchBook data in
## Start with companies that HAVE received VC funding. These are the 1 companies (VC investment =1)

In [485]:
vc_general_info_df = pd.read_excel("../data/raw/CA_VC_PitchBook/Company_General_Information.xlsx",header=6)
vc_general_info_df.head(2)
# interesting columns= Company ID (primary key), Description, Company Name, HQ Post Code, 
#Primary Industry Code, Primary Contact, Year Founded, Active Investors

,Company ID,Company Name,HQ Post Code,Description,Primary Industry Code,HQ Location,Ownership Status,Company Financing Status,Active Investors,Website,HQ Email,HQ Phone,Primary Contact,Primary Contact Title,Primary Contact Email,Primary Contact Phone,Year Founded,PitchBook Link
0,61743-97,ZypMedia,94102,Developer of an online advertisement platform ...,Social/Platform Software,"San Francisco, CA",Privately Held (backing),Venture Capital-Backed,"Sinclair Broadcast Group, Sinclair Digital Ven...",www.zypmedia.com,info@extend.tv,+1 (415) 894-9181,Mark Goldman,"Co-Founder, Board Member & Chief Executive Off...",mgoldman@extend.tv,+1 (415) 894-9181,2013.0,NaN
1,103815-64,Zype,10001,Developer of a cloud platform for video publis...,Social/Platform Software,"New York, NY",Privately Held (backing),Venture Capital-Backed,"Alpine Meridian, Berlin Ventures, Bill Dolan, ...",www.zype.com,contact@zype.com,NaN,Chris Bassolino,"Co-Founder, Chief Operating Officer & Director",NaN,NaN,2013.0,NaN


In [486]:
vc_general_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 18 columns):
Company ID                  1023 non-null object
Company Name                1022 non-null object
HQ Post Code                925 non-null object
Description                 1000 non-null object
Primary Industry Code       1000 non-null object
HQ Location                 998 non-null object
Ownership Status            1000 non-null object
Company Financing Status    1000 non-null object
Active Investors            966 non-null object
Website                     983 non-null object
HQ Email                    706 non-null object
HQ Phone                    705 non-null object
Primary Contact             980 non-null object
Primary Contact Title       980 non-null object
Primary Contact Email       871 non-null object
Primary Contact Phone       723 non-null object
Year Founded                970 non-null float64
PitchBook Link              0 non-null float64
dtypes: float64(2), obj

In [487]:
vc_last_financing_df = pd.read_excel("../data/raw/CA_VC_PitchBook/Last_Financing_Details.xlsx",header=6)
vc_last_financing_df .head(2)
# interesting columns = Company ID ( primary key), Company Name, Growth Rate, Size Multiple, last financing date,
# last financing Size, Last financing valuation, Last Financing Deal Type 2

# Note : Only want series A or later, filter OUT the seed rounds

,Company ID,Company Name,Revenue,Growth Rate,Size Multiple,Last Financing Date,Last Financing Deal Type,Last Financing Deal Type 2,Last Financing Size,Last Financing Valuation,...,Ownership Status,Company Financing Status,Financing Status Note,Active Investors,Acquirers,Primary Industry Code,Description,HQ Location,Website,PitchBook Link
0,61743-97,ZypMedia,NaN,0.000000,0.144068,2015-04-24,Early Stage VC,Series B,4.42,9.47,...,Privately Held (backing),Venture Capital-Backed,The company raised $4.42 million of Series B v...,"Sinclair Broadcast Group, Sinclair Digital Ven...",NaN,Social/Platform Software,Developer of an online advertisement platform ...,"San Francisco, CA",www.zypmedia.com,NaN
1,103815-64,Zype,NaN,0.919952,9.045216,2016-08-01,Seed Round,Seed,3.60,10.05,...,Privately Held (backing),Venture Capital-Backed,The company raised $3.6 million of seed ventur...,"Alpine Meridian, Berlin Ventures, Bill Dolan, ...",NaN,Social/Platform Software,Developer of a cloud platform for video publis...,"New York, NY",www.zype.com,NaN


In [488]:
vc_last_financing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 21 columns):
Company ID                     1023 non-null object
Company Name                   1022 non-null object
Revenue                        113 non-null float64
Growth Rate                    957 non-null float64
Size Multiple                  957 non-null float64
Last Financing Date            869 non-null datetime64[ns]
Last Financing Deal Type       994 non-null object
Last Financing Deal Type 2     585 non-null object
Last Financing Size            695 non-null float64
Last Financing Valuation       352 non-null float64
Last Financing Status          994 non-null object
Ownership Status               1000 non-null object
Company Financing Status       1000 non-null object
Financing Status Note          997 non-null object
Active Investors               966 non-null object
Acquirers                      1 non-null object
Primary Industry Code          1000 non-null object
Descriptio

In [489]:
vc_company_financials_df = pd.read_excel("../data/raw/CA_VC_PitchBook/Public_Company_Financials.xlsx",header=6)
vc_company_financials_df.head(2)
# Interesting columns are NOTHING

,Company ID,Company Name,Exchange,Ticker,Fiscal Period,Revenue,Gross Profit,EBITDA,Net Income,Enterprise Value,Ownership Status,Company Financing Status,Last Financing Date,Last Financing Deal Type,Last Financing Size,PitchBook Link
0,61743-97,ZypMedia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Privately Held (backing),Venture Capital-Backed,2015-04-24,Early Stage VC,4.42,NaN
1,103815-64,Zype,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Privately Held (backing),Venture Capital-Backed,2016-08-01,Seed Round,3.60,NaN


In [490]:
vc_company_financials_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 16 columns):
Company ID                  1023 non-null object
Company Name                1022 non-null object
Exchange                    1 non-null object
Ticker                      1 non-null object
Fiscal Period               122 non-null object
Revenue                     113 non-null float64
Gross Profit                7 non-null float64
EBITDA                      19 non-null float64
Net Income                  28 non-null float64
Enterprise Value            2 non-null float64
Ownership Status            1000 non-null object
Company Financing Status    1000 non-null object
Last Financing Date         869 non-null datetime64[ns]
Last Financing Deal Type    994 non-null object
Last Financing Size         695 non-null float64
PitchBook Link              0 non-null float64
dtypes: datetime64[ns](1), float64(7), object(8)
memory usage: 128.2+ KB


In [491]:
vc_social_web_df =  pd.read_excel("../data/raw/CA_VC_PitchBook/Social_and_Web_Presence.xlsx",header=6)
vc_social_web_df.head(2)
# interesting columns = company id (primary key), company name, growth rate, size multiple, majestic referring domains
# facebook likes, Tiwtter followers, Employees, Total raised

,Company ID,Company Name,Growth Rate,Size Multiple,Compete Unique Visitors,Majestic Referring Domains,Facebook Likes,Twitter Followers,Employees,Total Raised,Primary Industry Code,Description,Active Investors,Company Financing Status,Ownership Status,HQ Location,Website,PitchBook Link
0,61743-97,ZypMedia,0.000000,0.144068,NaN,NaN,NaN,51.0,12.0,6.82,Social/Platform Software,Developer of an online advertisement platform ...,"Sinclair Broadcast Group, Sinclair Digital Ven...",Venture Capital-Backed,Privately Held (backing),"San Francisco, CA",www.zypmedia.com,NaN
1,103815-64,Zype,0.919952,9.045216,NaN,188.0,1904.0,8352.0,11.0,3.60,Social/Platform Software,Developer of a cloud platform for video publis...,"Alpine Meridian, Berlin Ventures, Bill Dolan, ...",Venture Capital-Backed,Privately Held (backing),"New York, NY",www.zype.com,NaN


In [501]:
vc_social_web_df.tail()

,Company ID,Company Name,Growth Rate,Size Multiple,Compete Unique Visitors,Majestic Referring Domains,Facebook Likes,Twitter Followers,Employees,Total Raised,Primary Industry Code,Description,Active Investors,Company Financing Status,Ownership Status,HQ Location,Website,PitchBook Link
1020,95609-35,TransitScreen,0.135018,3.438100,NaN,65.0,774.0,3278.0,18.0,2.17,Application Software,Provider of a real-time transportation display...,"1776, Alex Bresler, BallstonBID LaunchPad, Bru...",Venture Capital-Backed,Privately Held (backing),"Washington, DC",www.transitscreen.com,NaN
1021,61382-89,Transifex,0.110816,365.999512,NaN,26806.0,2096.0,1870.0,72.0,6.70,Application Software,"Provider of a cloud-based, open-source transla...","Arafura Ventures, Georgios Papadopoulos, Ilya ...",Venture Capital-Backed,Privately Held (backing),"Menlo Park, CA",www.transifex.com,NaN
1022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1024,"© PitchBook Data, Inc. 2017",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Join these four dataframes into one

In [493]:
vc_general_info_colDrop_df = vc_general_info_df[["Company ID", "Description", "Company Name", "HQ Post Code", "Primary Industry Code",
                                         "Primary Contact", "Year Founded", "Active Investors","HQ Location"]]

In [494]:
vc_last_financing_colDrop_df =vc_last_financing_df[["Company ID", "Growth Rate", "Size Multiple",
                                    "Last Financing Date","Last Financing Size","Last Financing Valuation",
                                                   "Last Financing Deal Type 2 "]]

In [495]:
vc_social_web_colDrop_df  =vc_social_web_df [["Company ID",  "Growth Rate", 
                                              "Size Multiple", "Majestic Referring Domains",
        "Facebook Likes", "Twitter Followers", "Employees", "Total Raised"]]

In [496]:
final_vc_df = vc_general_info_colDrop_df.merge(vc_last_financing_colDrop_df, on='Company ID').merge(vc_social_web_colDrop_df, 
                                                                                     on='Company ID')

In [497]:
final_vc_df .info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1031 entries, 0 to 1030
Data columns (total 22 columns):
Company ID                     1023 non-null object
Description                    1000 non-null object
Company Name                   1022 non-null object
HQ Post Code                   925 non-null object
Primary Industry Code          1000 non-null object
Primary Contact                980 non-null object
Year Founded                   970 non-null float64
Active Investors               966 non-null object
HQ Location                    998 non-null object
Growth Rate_x                  957 non-null float64
Size Multiple_x                957 non-null float64
Last Financing Date            869 non-null datetime64[ns]
Last Financing Size            695 non-null float64
Last Financing Valuation       352 non-null float64
Last Financing Deal Type 2     585 non-null object
Growth Rate_y                  957 non-null float64
Size Multiple_y                957 non-null float64
Majesti

In [500]:
final_vc_df .tail(20)

,Company ID,Description,Company Name,HQ Post Code,Primary Industry Code,Primary Contact,Year Founded,Active Investors,HQ Location,Growth Rate_x,...,Last Financing Size,Last Financing Valuation,Last Financing Deal Type 2,Growth Rate_y,Size Multiple_y,Majestic Referring Domains,Facebook Likes,Twitter Followers,Employees,Total Raised
1011,167939-56,Manufacturer of automated strawberry harvestin...,Traptic,94043,Machinery (B2B),Lewis Anderson,2016.0,NaN,"Mountain View, CA",NaN,...,0.40,2.0,Seed,NaN,NaN,NaN,NaN,NaN,NaN,0.40
1012,53592-49,Provider of an online platform designed to hel...,Trapit,94107,Media and Information Services (B2B),Henry Nothhaft,2010.0,"Horizons Ventures, Li Ka-Shing, Rogers Venture...","San Francisco, CA",-0.001348,...,10.00,NaN,Series B,-0.001348,5.368106,142.0,988.0,4431.0,33.0,24.78
1013,90367-21,Developer of a transactional data technology. ...,Tranzlogic,93065,Other IT Services,Charles Hogan,2011.0,SJ Strategic Investments,"Simi Valley, CA",-0.159625,...,1.00,NaN,NaN,-0.159625,0.427902,14.0,91.0,297.0,11.0,2.70
1014,172917-91,Provider of an all-in-one virtual reality plat...,Transported,NaN,Application Software,Tyson Woeste,NaN,Corigin Ventures,"Los Angeles, CA",1.212312,...,NaN,NaN,Seed,1.212312,1.697015,107.0,192.0,118.0,NaN,NaN
1015,51622-39,Developer of gallium nitride power devices des...,Transphorm,93117,Application Specific Semiconductors,Cameron McAulay,2007.0,"Bright Capital, Bright Capital Partners, Found...","Goleta, CA",0.508492,...,NaN,NaN,NaN,0.508492,5.068752,350.0,158.0,390.0,105.0,219.05
1016,52893-37,Provider of check and eCheck payment provision...,Transmodus,93036,Business/Productivity Software,Robert McShirley,2001.0,Tech Coast Angels,"Oxnard, CA",-0.062313,...,NaN,NaN,NaN,-0.062313,0.483776,15.0,NaN,199.0,14.0,1.00
1017,58446-37,Provider of a navigation system for reducing X...,Translucent Medical,95060,Other Healthcare Technology Systems,Dan Wallace,2011.0,NaN,"Santa Cruz, CA",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1018,90366-58,Provider of a cloud-based platform for continu...,Translation Exchange,94105,Social/Platform Software,Michael Berkovich,2014.0,"Amplify.LA, Canyon Creek Capital, HillsVen Cap...","San Francisco, CA",3.822225,...,NaN,NaN,NaN,3.822225,0.555332,8.0,200.0,506.0,11.0,2.00
1019,44158-96,Manufacturer of semiconductor testing equipmen...,Translarity,94538,Other Semiconductors,Mark Gardiner,2005.0,"Applied Ventures, Intel Capital, KT Venture Gr...","Fremont, CA",0.000000,...,5.29,NaN,NaN,0.000000,0.216216,8.0,NaN,NaN,11.0,35.27
1020,95609-35,Provider of a real-time transportation display...,TransitScreen,20005,Application Software,Matt Caywood,2013.0,"1776, Alex Bresler, BallstonBID LaunchPad, Bru...","Washington, DC",0.135018,...,0.70,12.0,NaN,0.135018,3.438100,65.0,774.0,3278.0,18.0,2.17


In [434]:
final_vc_df .drop([
                  'Growth Rate_y','Size Multiple_y'],axis=1,inplace=True)

In [435]:
final_vc_df.rename(columns={'Growth Rate_x':'Growth Rate',"Size Multiple_x":'Size Multiple',
                           "Company Name_x":"Company Name"},inplace=True) # rename cols

In [436]:
final_vc_df.head(2)

,Company ID,Description,Company Name,HQ Post Code,Primary Industry Code,Primary Contact,Year Founded,Active Investors,HQ Location,Growth Rate,Size Multiple,Last Financing Date,Last Financing Size,Last Financing Valuation,Last Financing Deal Type 2,Majestic Referring Domains,Facebook Likes,Twitter Followers,Employees,Total Raised
0,61743-97,Developer of an online advertisement platform ...,ZypMedia,94102,Social/Platform Software,Mark Goldman,2013.0,"Sinclair Broadcast Group, Sinclair Digital Ven...","San Francisco, CA",0.000000,0.144068,2015-04-24,4.42,9.47,Series B,NaN,NaN,51.0,12.0,6.82
1,103815-64,Developer of a cloud platform for video publis...,Zype,10001,Social/Platform Software,Chris Bassolino,2013.0,"Alpine Meridian, Berlin Ventures, Bill Dolan, ...","New York, NY",0.919952,9.045216,2016-08-01,3.60,10.05,Seed,188.0,1904.0,8352.0,11.0,3.60


### Filter out the seed rounds
- Only want Series A

In [211]:
final_vc_df['Last Financing Deal Type 2 '].unique()

array(['Series B', 'Seed', 'Series B2', nan, 'Series A', 'Series B1',
       'Series D', 'Series C', 'Series A1', 'Series E', 'Series A2',
       'Angel', 'Corporate', 'Senior Debt', 'Series 2', 'Series 1',
       'Add-on', 'Loan', 'Series CC', 'Series F', 'Acquisition Financing',
       'Series G', 'Revolving Credit Line', 'Series C3', 'Series 3',
       'Series AA', 'Series B3', 'Subordinated Debt', 'Series C1'], dtype=object)

In [437]:
final_vc_financeTypeFilter_df = final_vc_df.loc[(final_vc_df['Last Financing Deal Type 2 ']!='Seed') & 
                (final_vc_df['Last Financing Deal Type 2 ']!='Angel') &
                 (final_vc_df['Last Financing Deal Type 2 ']!='Series B')& (final_vc_df['Last Financing Deal Type 2 ']!='Series B2')   &
                            (final_vc_df['Last Financing Deal Type 2 ']!='Series B1') & (final_vc_df['Last Financing Deal Type 2 ']!='Series D') &
    (final_vc_df['Last Financing Deal Type 2 ']!='Series C') & (final_vc_df['Last Financing Deal Type 2 ']!='Series E') &
 (final_vc_df['Last Financing Deal Type 2 ']!='Series 2') &  (final_vc_df['Last Financing Deal Type 2 ']!='Series CC') &
    (final_vc_df['Last Financing Deal Type 2 ']!='Series F') & (final_vc_df['Last Financing Deal Type 2 ']!='Series G') &
    (final_vc_df['Last Financing Deal Type 2 ']!='Series C3') &    (final_vc_df['Last Financing Deal Type 2 ']!='Series 3') & 
    (final_vc_df['Last Financing Deal Type 2 ']!='Series B3') & (final_vc_df['Last Financing Deal Type 2 ']!='Series C1'),
                                                : ]

In [438]:
final_vc_financeTypeFilter_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 639 entries, 4 to 1030
Data columns (total 20 columns):
Company ID                     631 non-null object
Description                    608 non-null object
Company Name                   630 non-null object
HQ Post Code                   558 non-null object
Primary Industry Code          608 non-null object
Primary Contact                594 non-null object
Year Founded                   592 non-null float64
Active Investors               588 non-null object
HQ Location                    606 non-null object
Growth Rate                    581 non-null float64
Size Multiple                  581 non-null float64
Last Financing Date            491 non-null datetime64[ns]
Last Financing Size            365 non-null float64
Last Financing Valuation       144 non-null float64
Last Financing Deal Type 2     193 non-null object
Majestic Referring Domains     540 non-null float64
Facebook Likes                 441 non-null float64
Twitter Foll

In [439]:
final_vc_financeTypeFilter_df.head(2)

,Company ID,Description,Company Name,HQ Post Code,Primary Industry Code,Primary Contact,Year Founded,Active Investors,HQ Location,Growth Rate,Size Multiple,Last Financing Date,Last Financing Size,Last Financing Valuation,Last Financing Deal Type 2,Majestic Referring Domains,Facebook Likes,Twitter Followers,Employees,Total Raised
4,52852-15,Provider of a channel data management cloud pl...,Zyme,94065,Media and Information Services (B2B),Chandran Sankaran,2004.0,"Artiman Ventures, Asiabio Zyme Solutions, Susq...","Redwood City, CA",4.139861,5.683374,2017-04-12,100.0,100.0,NaN,330.0,197.0,1300.0,501.0,13.1
5,108882-55,Provider of a B2B networking and routing platf...,Zycada Networks,95035,Application Software,Subbu Varadarajan,2014.0,"Alchemist Accelerator, Cervin Ventures, Chris ...","Milpitas, CA",0.000000,0.298939,2015-12-15,2.5,8.0,Series A,17.0,NaN,49.0,7.0,2.5


# Drop companies missing the zip code, year founded, and primary contact
- Can't impute this

In [440]:
final_vc_dropFinanceZipYear_df = final_vc_financeTypeFilter_df.loc[
    (final_vc_financeTypeFilter_df['HQ Post Code'].isnull()==False) &
    (final_vc_financeTypeFilter_df['Year Founded'].isnull()==False) &
    (final_vc_financeTypeFilter_df['Primary Contact'].isnull()==False),: ]

In [441]:
final_vc_dropFinanceZipYear_df .info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 536 entries, 4 to 1021
Data columns (total 20 columns):
Company ID                     536 non-null object
Description                    536 non-null object
Company Name                   536 non-null object
HQ Post Code                   536 non-null object
Primary Industry Code          536 non-null object
Primary Contact                536 non-null object
Year Founded                   536 non-null float64
Active Investors               519 non-null object
HQ Location                    536 non-null object
Growth Rate                    518 non-null float64
Size Multiple                  518 non-null float64
Last Financing Date            441 non-null datetime64[ns]
Last Financing Size            332 non-null float64
Last Financing Valuation       129 non-null float64
Last Financing Deal Type 2     169 non-null object
Majestic Referring Domains     481 non-null float64
Facebook Likes                 403 non-null float64
Twitter Foll

In [442]:
final_vc_dropFinanceZipYear_df .describe()

,Year Founded,Growth Rate,Size Multiple,Last Financing Size,Last Financing Valuation,Majestic Referring Domains,Facebook Likes,Twitter Followers,Employees,Total Raised
count,536.000000,518.000000,518.000000,332.000000,129.000000,481.000000,4.030000e+02,430.000000,459.000000,419.000000
mean,2009.166045,0.429057,86.065641,22.437108,97.726434,1945.316008,2.197005e+05,13265.686047,133.925926,58.695943
std,8.002362,1.268712,710.299617,143.922757,338.758646,20549.204663,2.280783e+06,73605.080865,761.532375,718.808544
min,1903.000000,-4.380726,0.007453,0.010000,0.170000,1.000000,1.000000e+00,1.000000,1.000000,0.020000
25%,2007.000000,0.000000,0.608843,1.015000,10.320000,23.000000,1.630000e+02,164.250000,9.000000,2.000000
50%,2011.000000,0.022076,2.305364,4.000000,21.850000,83.000000,8.670000e+02,807.500000,18.000000,6.600000
75%,2013.000000,0.415943,8.207944,8.000000,37.630000,270.000000,7.053000e+03,3340.000000,51.000000,19.750000
max,2016.000000,13.422807,13063.853941,2250.000000,2600.000000,440467.000000,4.187224e+07,769227.000000,12000.000000,14688.060000


# Impute missing values
- Growth Rate : impute median
- Size Multiple: imput median
- Last Financing Date : dont't really need this ( only need for angel companies)
- Last Financing Size : impute median
- Last Financing Valuation: Drop this ( too many nulls)
- Majestic Referring Domains: Impute Median
- Facebook Like: Impute median
- Twitter followers: impute median
- Employees: impute median
- Total raised: impute median

In [444]:
# first drop last financing valuation (not enough data)
final_vc_dropFinanceZipYear_df.drop(['Last Financing Valuation'],axis=1,inplace=True)

ValueError: labels ['Last Financing Valuation'] not contained in axis

In [447]:
median_values={}
for row in final_vc_dropFinanceZipYear_df.describe(): # get median values
    if row =='Last Financing Valuation': # don't have enough data for this
        pass
    else:
        median_values[row]=final_vc_dropFinanceZipYear_df.describe()[row]["50%"]

In [448]:
median_values

{'Employees': 18.0,
 'Facebook Likes': 867.0,
 'Growth Rate': 0.022076193029139247,
 'Last Financing Size': 4.0,
 'Majestic Referring Domains': 83.0,
 'Size Multiple': 2.3053636869314835,
 'Total Raised': 6.5999999999999996,
 'Twitter Followers': 807.5,
 'Year Founded': 2011.0}

In [449]:
imputed_final_df = final_vc_dropFinanceZipYear_df.copy()
for key in median_values: # update the nan values with the median
    updated_col = final_vc_dropFinanceZipYear_df.loc[:,key].copy()
    updated_col = updated_col.fillna(median_values[key])
    imputed_final_df.loc[:,key] = updated_col


In [450]:
imputed_final_df.head(2)

,Company ID,Description,Company Name,HQ Post Code,Primary Industry Code,Primary Contact,Year Founded,Active Investors,HQ Location,Growth Rate,Size Multiple,Last Financing Date,Last Financing Size,Last Financing Deal Type 2,Majestic Referring Domains,Facebook Likes,Twitter Followers,Employees,Total Raised
4,52852-15,Provider of a channel data management cloud pl...,Zyme,94065,Media and Information Services (B2B),Chandran Sankaran,2004.0,"Artiman Ventures, Asiabio Zyme Solutions, Susq...","Redwood City, CA",4.139861,5.683374,2017-04-12,100.0,NaN,330.0,197.0,1300.0,501.0,13.1
5,108882-55,Provider of a B2B networking and routing platf...,Zycada Networks,95035,Application Software,Subbu Varadarajan,2014.0,"Alchemist Accelerator, Cervin Ventures, Chris ...","Milpitas, CA",0.000000,0.298939,2015-12-15,2.5,Series A,17.0,867.0,49.0,7.0,2.5


### Add a 1 for the column VC_Invested

In [451]:
imputed_final_df['VC_invested']=1

In [452]:
imputed_final_df.head(20)

,Company ID,Description,Company Name,HQ Post Code,Primary Industry Code,Primary Contact,Year Founded,Active Investors,HQ Location,Growth Rate,Size Multiple,Last Financing Date,Last Financing Size,Last Financing Deal Type 2,Majestic Referring Domains,Facebook Likes,Twitter Followers,Employees,Total Raised,VC_invested
4,52852-15,Provider of a channel data management cloud pl...,Zyme,94065,Media and Information Services (B2B),Chandran Sankaran,2004.0,"Artiman Ventures, Asiabio Zyme Solutions, Susq...","Redwood City, CA",4.139861,5.683374,2017-04-12,100.00,NaN,330.0,197.0,1300.0,501.0,13.10,1
5,108882-55,Provider of a B2B networking and routing platf...,Zycada Networks,95035,Application Software,Subbu Varadarajan,2014.0,"Alchemist Accelerator, Cervin Ventures, Chris ...","Milpitas, CA",0.000000,0.298939,2015-12-15,2.50,Series A,17.0,867.0,49.0,7.0,2.50,1
6,62229-34,"Developer of animated, interactive learning ma...",Zyante,95031,Publishing,Smita Bakshi,2012.0,"Bialla Venture Partners, National Science Foun...","Los Gatos, CA",0.078953,0.760068,2016-02-10,4.00,Series A,83.0,612.0,267.0,32.0,4.00,1
7,52214-95,Developer of music creation software designed ...,Zya,91302,Multimedia and Design Software,Alberto Lomeli,2008.0,"Individual Investor, Intel Capital, Liberty Gl...","Calabasas, CA",0.192032,3.086235,2016-01-12,4.26,NaN,123.0,2269.0,807.5,51.0,44.86,1
8,117491-32,Provider of interactive fitness entertainment ...,Zwift,90802,Entertainment Software,Eric Min,2014.0,"COLOPL, Max Levchin, Novator Partners, Samchul...","Long Beach, CA",0.143940,73.107156,2016-11-16,27.00,Series A,471.0,158746.0,24569.0,70.0,45.00,1
10,103593-34,Developer of a mobile-based browsing applicati...,Zurf,94107,Social/Platform Software,Shahin Shadfar,2013.0,"Bobby Yazdani, Geekdom SF, Signatures Capital","San Francisco, CA",-0.088942,0.479786,NaT,4.00,NaN,25.0,166.0,127.0,6.0,6.60,1
11,51251-95,Provider of an on-demand subscription billing ...,Zuora,94404,Other Software,Tyler Sloat,2007.0,"Benchmark Capital, BlackRock Private Equity Pa...","Foster City, CA",0.179027,41.123341,2016-06-01,4.00,NaN,2031.0,3827.0,17692.0,631.0,250.00,1
14,148845-52,Provider of pizza services with help of robots...,Zume Pizza,94043,Food Products,Julia Collins,2015.0,"AME Cloud Ventures, Kortschak Investments, Mav...","Mountain View, CA",1.171114,3.110912,2016-12-07,22.69,NaN,155.0,1986.0,562.0,18.0,22.69,1
16,104840-56,Provider of services in order to help startup ...,Zuma Ventures,90401,Other Commercial Services,David Carter,2014.0,"Aaron Hirschhorn, Burns Israelsen, Derek Smith...","Santa Monica, CA",0.131263,0.596335,2014-08-28,0.10,NaN,18.0,572.0,246.0,15.0,0.10,1
20,125763-49,Provider of an employee feedback platform. The...,Zugata,94306,Social/Platform Software,Philippe Van Nuijs,2014.0,"Canaan Partners, Formation 8, General Catalyst...","Palo Alto, CA",0.408153,2.269011,2016-11-14,7.00,Series A,140.0,867.0,267.0,8.0,10.20,1


# Tweepy to find usernames of the people

In [467]:
def username_search(name, company, state, c = 20):
    """Run a search on twitter for the given name. Returns the first username (should be the most relevant).
    Looks to match a state location with the state locatio nof the company

    First try searching for the person's name + company. If that does not work, try just searching for the 
    person's name.
    
    Count is for the number of results to return. Default to twenty. If not in the first twenty results, probably not
    the correct user"""
    state = state.lower()
    
    credentials = yaml.load(open(os.path.expanduser('~/.ssh/api_credentials.yml')))
    auth = tweepy.OAuthHandler(credentials['twitter']['consumer_key'], credentials['twitter']['consumer_secret'],)
    auth.set_access_token(credentials['twitter']['token'], credentials['twitter']['token_secret'])
    api = tweepy.API(auth,wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    

    try: # search the name and the company
        tweets = api.search_users(q=str(name)+" "+str(company), count=c)
        test = result[0].screen_name
        screen_n = None
        
        for result in tweets:
            location = result.location.lower().split(" ") # see if the location is in the companies state
            if state in location:
                return result.screen_name

        if screen_n == None:
            return 'NaN'
        else:
            return screen_n
            
    except Exception as e: # try just the name
        try:

            tweets = api.search_users(q=name, count = c)
            
            screen_n = None
            for result in tweets:
                if state in result.location.lower().split(" "):

                    return result.screen_name
            if screen_n == None:
                return "NaN"
            else:
                return screen_n
            

        except Exception as e:
            return "NaN"

### Test the accuracy of this api for finding usernames


In [454]:
test_names = ['Jonathan Hilgart None NJ','Tim Cook Apple CA', "Smita Bakshi Zyante CA","Rob Fuggetta zuberance CA",
              "Charles Hogan tranzlogic CA", 
              "Duke Chung travelbank CA", "Jake Fields treeline CA","Eric Min Zwift CA","Dominic Lewis Transifex CA",
             "Dimitris Glezos Transifex CA"]
names_username = {"Jonathan Hilgart":"topofthehil",'Tim Cook':"tim_cook", "Smita Bakshi":"sbaksh",
                 "Rob Fuggetta":"robfuggetta", "Charles Hogan":"Tranzlogic",
                 "Duke Chung":"Duke_Chung", "Jake Fields":"JakeFields","Eric Min":"werkdodger",
                  "Dominic Lewis":"domwlewis", " Dominic Lewis":"glezos"}

In [455]:
test_search= {}
for name in test_names:
    first,last,company,state = name.split(" ")
    result = username_search(first+" "+last,company,state)
    print(result)
    test_search[name]=result

NaN
CoachCook4444
sbaksh
NaN
NaN
Duke_Chung
JakeFields
notEricMin
NaN
NaN


In [456]:
username_intersection = set(test_search.values()).intersection(set(names_username.values()))

In [457]:
len(username_intersection)/len(names_username.keys())

0.3

In [458]:
"ca" in "hickory, north carolina usa  location"

True

In [459]:
test_search

{'Charles Hogan tranzlogic CA': 'NaN',
 'Dimitris Glezos Transifex CA': 'NaN',
 'Dominic Lewis Transifex CA': 'NaN',
 'Duke Chung travelbank CA': 'Duke_Chung',
 'Eric Min Zwift CA': 'notEricMin',
 'Jake Fields treeline CA': 'JakeFields',
 'Jonathan Hilgart None NJ': 'NaN',
 'Rob Fuggetta zuberance CA': 'NaN',
 'Smita Bakshi Zyante CA': 'sbaksh',
 'Tim Cook Apple CA': 'CoachCook4444'}

#### Decent! About 50% accuracy in finding twitter handles

### Add the Twitter username to the pandas df for the VC dataframe

In [468]:
twitter_usernames_vc_df = []
for idx,row in enumerate(imputed_final_df.iterrows()):
    location = row[1]['HQ Location'].split(",")[1].strip(" ")
    company = row[1]['Company Name']
    founder = row[1]['Primary Contact']

    twitter_usernames_vc_df.append(username_search(founder,company, location ))
    if idx%100 ==0:
        print(f"Finished {idx/len(imputed_final_df)}")


Finished 0.0
Rate limit reached. Sleeping for: 355
Finished 0.1865671641791045
Finished 0.373134328358209
Finished 0.5597014925373134
Finished 0.746268656716418
Rate limit reached. Sleeping for: 68
Finished 0.9328358208955224


In [469]:
imputed_final_df['Twitter_Username'] = twitter_usernames_vc_df

In [484]:
imputed_final_df.tail()

,Company ID,Description,Company Name,HQ Post Code,Primary Industry Code,Primary Contact,Year Founded,Active Investors,HQ Location,Growth Rate,...,Last Financing Date,Last Financing Size,Last Financing Deal Type 2,Majestic Referring Domains,Facebook Likes,Twitter Followers,Employees,Total Raised,VC_invested,Twitter_Username
1017,58446-37,Provider of a navigation system for reducing X...,Translucent Medical,95060,Other Healthcare Technology Systems,Dan Wallace,2011.0,NaN,"Santa Cruz, CA",0.022076,...,NaT,4.00,NaN,83.0,867.0,807.5,18.0,6.60,1,WeLiveNThMatrix
1018,90366-58,Provider of a cloud-based platform for continu...,Translation Exchange,94105,Social/Platform Software,Michael Berkovich,2014.0,"Amplify.LA, Canyon Creek Capital, HillsVen Cap...","San Francisco, CA",3.822225,...,2016-03-01,4.00,NaN,8.0,200.0,506.0,11.0,2.00,1,micberk
1019,44158-96,Manufacturer of semiconductor testing equipmen...,Translarity,94538,Other Semiconductors,Mark Gardiner,2005.0,"Applied Ventures, Intel Capital, KT Venture Gr...","Fremont, CA",0.000000,...,2014-07-30,5.29,NaN,8.0,867.0,807.5,11.0,35.27,1,NaN
1020,95609-35,Provider of a real-time transportation display...,TransitScreen,20005,Application Software,Matt Caywood,2013.0,"1776, Alex Bresler, BallstonBID LaunchPad, Bru...","Washington, DC",0.135018,...,2017-02-24,0.70,NaN,65.0,774.0,3278.0,18.0,2.17,1,MattCaywood
1021,61382-89,"Provider of a cloud-based, open-source transla...",Transifex,94025,Application Software,Dimitris Glezos,2009.0,"Arafura Ventures, Georgios Papadopoulos, Ilya ...","Menlo Park, CA",0.110816,...,NaT,4.00,NaN,26806.0,2096.0,1870.0,72.0,6.70,1,NaN


## Drop NaN Usernames
- Also, drop last financing Date and last financing deal type 2 (don't need for the VC round)

In [471]:
final_acce_incub_df =  imputed_final_df[
        (imputed_final_df.Twitter_Username!='NaN') ]

In [473]:
final_acce_incub_df .head()

,Company ID,Description,Company Name,HQ Post Code,Primary Industry Code,Primary Contact,Year Founded,Active Investors,HQ Location,Growth Rate,...,Last Financing Date,Last Financing Size,Last Financing Deal Type 2,Majestic Referring Domains,Facebook Likes,Twitter Followers,Employees,Total Raised,VC_invested,Twitter_Username
6,62229-34,"Developer of animated, interactive learning ma...",Zyante,95031,Publishing,Smita Bakshi,2012.0,"Bialla Venture Partners, National Science Foun...","Los Gatos, CA",0.078953,...,2016-02-10,4.00,Series A,83.0,612.0,267.0,32.0,4.00,1,sbaksh
8,117491-32,Provider of interactive fitness entertainment ...,Zwift,90802,Entertainment Software,Eric Min,2014.0,"COLOPL, Max Levchin, Novator Partners, Samchul...","Long Beach, CA",0.143940,...,2016-11-16,27.00,Series A,471.0,158746.0,24569.0,70.0,45.00,1,notEricMin
10,103593-34,Developer of a mobile-based browsing applicati...,Zurf,94107,Social/Platform Software,Shahin Shadfar,2013.0,"Bobby Yazdani, Geekdom SF, Signatures Capital","San Francisco, CA",-0.088942,...,NaT,4.00,NaN,25.0,166.0,127.0,6.0,6.60,1,sshadfar
14,148845-52,Provider of pizza services with help of robots...,Zume Pizza,94043,Food Products,Julia Collins,2015.0,"AME Cloud Ventures, Kortschak Investments, Mav...","Mountain View, CA",1.171114,...,2016-12-07,22.69,NaN,155.0,1986.0,562.0,18.0,22.69,1,JuliaRivka
16,104840-56,Provider of services in order to help startup ...,Zuma Ventures,90401,Other Commercial Services,David Carter,2014.0,"Aaron Hirschhorn, Burns Israelsen, Derek Smith...","Santa Monica, CA",0.131263,...,2014-08-28,0.10,NaN,18.0,572.0,246.0,15.0,0.10,1,_davidcarter


In [476]:
final_acce_incub_df .info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128 entries, 6 to 1020
Data columns (total 21 columns):
Company ID                     128 non-null object
Description                    128 non-null object
Company Name                   128 non-null object
HQ Post Code                   128 non-null object
Primary Industry Code          128 non-null object
Primary Contact                128 non-null object
Year Founded                   128 non-null float64
Active Investors               125 non-null object
HQ Location                    128 non-null object
Growth Rate                    128 non-null float64
Size Multiple                  128 non-null float64
Last Financing Date            113 non-null datetime64[ns]
Last Financing Size            128 non-null float64
Last Financing Deal Type 2     39 non-null object
Majestic Referring Domains     128 non-null float64
Facebook Likes                 128 non-null float64
Twitter Followers              128 non-null float64
Employees    

In [480]:
final_acce_incub_df.to_csv("../data/processed/PitchBook_CA_VCInvest=1.csv")

In [481]:
t = pd.read_csv("../data/processed/PitchBook_CA_VCInvest=1.csv")

In [483]:
final_acce_incub_df.tail()

,Company ID,Description,Company Name,HQ Post Code,Primary Industry Code,Primary Contact,Year Founded,Active Investors,HQ Location,Growth Rate,...,Last Financing Date,Last Financing Size,Last Financing Deal Type 2,Majestic Referring Domains,Facebook Likes,Twitter Followers,Employees,Total Raised,VC_invested,Twitter_Username
1008,106663-87,Developer of an online platform designed to bo...,Traveling Spoon,94110,"Other Restaurants, Hotels and Leisure",Aashi Vel,2013.0,"Anjan Mitra, Dorm Room Fund, Emily Mitra, Erik...","San Francisco, CA",0.416151,...,2015-09-17,0.87,NaN,237.0,19126.0,942.0,2.0,1.03,1,TravelingSpoon
1009,167364-55,Developer of travel and expense software for s...,TravelBank,94107,Application Software,Duke Chung,2015.0,"Accel, New Enterprise Associates","San Francisco, CA",4.039472,...,2016-10-20,10.00,Series A,43.0,916.0,313.0,11.0,10.00,1,Duke_Chung
1017,58446-37,Provider of a navigation system for reducing X...,Translucent Medical,95060,Other Healthcare Technology Systems,Dan Wallace,2011.0,NaN,"Santa Cruz, CA",0.022076,...,NaT,4.00,NaN,83.0,867.0,807.5,18.0,6.60,1,WeLiveNThMatrix
1018,90366-58,Provider of a cloud-based platform for continu...,Translation Exchange,94105,Social/Platform Software,Michael Berkovich,2014.0,"Amplify.LA, Canyon Creek Capital, HillsVen Cap...","San Francisco, CA",3.822225,...,2016-03-01,4.00,NaN,8.0,200.0,506.0,11.0,2.00,1,micberk
1020,95609-35,Provider of a real-time transportation display...,TransitScreen,20005,Application Software,Matt Caywood,2013.0,"1776, Alex Bresler, BallstonBID LaunchPad, Bru...","Washington, DC",0.135018,...,2017-02-24,0.70,NaN,65.0,774.0,3278.0,18.0,2.17,1,MattCaywood


In [ ]:
# 0, 1 ,1 , 1, 0 - amir, 1, 1, 1 , .5, 0 ,1 

In [ ]:
# ~ 65% accuracy

In [ ]:
 # 68% acuracy7.5/11